In [27]:
# !pip install pulp

In [28]:
import pulp # Library to perfom linear programming
from pulp import LpProblem

In [38]:
from itertools import combinations

In [43]:
times = {
    'Swimmer 1': {'Backstroke': 69, 'Breaststroke': 72, 'Butterfly': 63, 'Freestyle': 59},
    'Swimmer 2': {'Backstroke': 66, 'Breaststroke': 71, 'Butterfly': 65, 'Freestyle': 58},
    'Swimmer 3': {'Backstroke': 68, 'Breaststroke': 70, 'Butterfly': 67, 'Freestyle': 55},
    'Swimmer 4': {'Backstroke': 67, 'Breaststroke': 75, 'Butterfly': 70, 'Freestyle': 57},
    'Swimmer 5': {'Backstroke': 72, 'Breaststroke': 69, 'Butterfly': 76, 'Freestyle': 58},
    'Swimmer 6': {'Backstroke': 68, 'Breaststroke': 73, 'Butterfly': 66, 'Freestyle': 56},
}

# List of all swimmers
all_swimmers = ['Swimmer 1', 'Swimmer 2', 'Swimmer 3', 'Swimmer 4', 'Swimmer 5', 'Swimmer 6']

# Strokes
strokes = ['Backstroke', 'Breaststroke', 'Butterfly', 'Freestyle']

# Initialize best result
best_cost = float('inf')
best_assignment = None

In [42]:
# Try all combinations of 4 swimmers from 6
for selected_swimmers in combinations(all_swimmers, 4):
    # Create a linear programming problem
    prob = pulp.LpProblem(f"Medley_Relay_Optimization_with_{selected_swimmers}", pulp.LpMinimize)

    # Create decision variables for the selected swimmers
    assignments = pulp.LpVariable.dicts("assign", (selected_swimmers, strokes), cat='Binary')

    # Objective function: Minimize total relay time
    prob += pulp.lpSum(assignments[swimmer][stroke] * times[swimmer][stroke]
                       for swimmer in selected_swimmers
                       for stroke in strokes)

    # Constraints: Each stroke should be assigned exactly one swimmer
    for stroke in strokes:
        prob += pulp.lpSum(assignments[swimmer][stroke] for swimmer in selected_swimmers) == 1

    # Constraints: Each swimmer is assigned to exactly one stroke
    for swimmer in selected_swimmers:
        prob += pulp.lpSum(assignments[swimmer][stroke] for stroke in strokes) == 1

    # Solve the problem
    status = prob.solve()

    # Check if this assignment is better
    current_cost = pulp.value(prob.objective)
    if current_cost < best_cost:
        best_cost = current_cost
        best_assignment = {swimmer: stroke for swimmer in selected_swimmers
                           for stroke in strokes
                           if pulp.value(assignments[swimmer][stroke]) == 1}

print(f"status: {prob.status}, {pulp.LpStatus[prob.status]}")

# Print the results
if best_assignment:
    print("Optimal assignment:")
    for swimmer, stroke in best_assignment.items():
        print(f"{swimmer} should swim {stroke}")
    print(f"Total minimum time: {best_cost} seconds")
else:
    print("No feasible solution found.")

status: 1, Optimal
Optimal assignment:
Swimmer 1 should swim Butterfly
Swimmer 2 should swim Backstroke
Swimmer 3 should swim Freestyle
Swimmer 5 should swim Breaststroke
Total minimum time: 253.0 seconds
